In [23]:
import numpy
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset

In [24]:
cuda_gpu = torch.cuda.is_available()
if (cuda_gpu):
    print("Great, I have a GPU!")
else:
    print("Life is short -- consider a GPU!")

Great, I have a GPU!


In [25]:
# Build CustomedDataset
# Separate the given dataset into two parts, four for train and one for test

class CustomedDataSet(Dataset):
    def __init__(self, train=True):
        self.train = train
        if self.train :
            trainX = pd.read_csv('./train.csv',nrows=33600)
            trainY = trainX.label.as_matrix().tolist()
            trainX = trainX.drop('label',axis=1).as_matrix().reshape(trainX.shape[0], 1, 28, 28)
            self.datalist = trainX
            self.labellist = trainY
        else:
            testX = pd.read_csv('./train.csv')
            testX = testX.drop(testX.head(33600).index)
            testY = testX.label.as_matrix().tolist()
            testX = testX.drop('label',axis=1).as_matrix().reshape(testX.shape[0], 1, 28, 28)
            self.datalist = testX
            self.labellist = testY
            
    def __getitem__(self, index):
        if self.train:
            return torch.Tensor(self.datalist[index].astype(float)),self.labellist[index]
        else:
            return torch.Tensor(self.datalist[index].astype(float)),self.labellist[index]
    
    def __len__(self):
        return self.datalist.shape[0]

In [26]:
batch_size = 100

In [27]:
train_dataset = CustomedDataSet()
test_dataset  = CustomedDataSet(train=False)
train_loader  = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True,num_workers=2)
test_loader   = DataLoader(dataset= test_dataset,batch_size=batch_size,shuffle=False)

/home/karry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
/home/karry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
/home/karry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/home/karry/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [28]:
# CNN Model

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1 ,16, kernel_size=5,padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5,padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [29]:
cnn = CNN()

In [30]:
# Hyper Parameters

num_epochs = 5
learning_rate = 0.001

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(),lr=learning_rate)

In [32]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        if cuda_gpu:
            images, labels = images.cuda(), labels.cuda()
            cnn.cuda()
            
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 112 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' %(epoch+1, num_epochs, i+1, 
                                                              len(train_dataset)//batch_size, loss.item()))

Epoch [1/5], Iter [112/336] Loss: 0.1491
Epoch [1/5], Iter [224/336] Loss: 0.1451
Epoch [1/5], Iter [336/336] Loss: 0.1407
Epoch [2/5], Iter [112/336] Loss: 0.0324
Epoch [2/5], Iter [224/336] Loss: 0.0588
Epoch [2/5], Iter [336/336] Loss: 0.1014
Epoch [3/5], Iter [112/336] Loss: 0.0772
Epoch [3/5], Iter [224/336] Loss: 0.0057
Epoch [3/5], Iter [336/336] Loss: 0.0107
Epoch [4/5], Iter [112/336] Loss: 0.0138
Epoch [4/5], Iter [224/336] Loss: 0.0163
Epoch [4/5], Iter [336/336] Loss: 0.0631
Epoch [5/5], Iter [112/336] Loss: 0.0103
Epoch [5/5], Iter [224/336] Loss: 0.0284
Epoch [5/5], Iter [336/336] Loss: 0.0124


In [33]:
cnn.eval() 

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

In [34]:
ans = torch.LongTensor()    # Build a tensor to concatenate answers.

In [35]:
for (images,labels) in test_loader:
    if cuda_gpu:
        images, labels = images.cuda(), labels.cuda()
        cnn.cuda()
        
    images = Variable(images)
    outputs = cnn(images)
    _,predicted = torch.max(outputs.data, 1)
    ans = torch.cat((ans,predicted.cpu()),0)

In [36]:
ans = ans.numpy()

In [37]:
corr=0
for i,(images,labels) in enumerate(test_dataset):
    if ans[i]==labels:
        corr=corr+1
print("%.3F"%(corr/84))    # Our final accuracy over 8400 digits.

98.786
